In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P, Utilities as U
from mppi.Calculators import Tools
from mppi.Datasets import PostProcessing as PP
from mppi.Utilities import Constants as Const
from mppi.Utilities import LatticeUtils as LL
import matplotlib.pyplot as plt
import numpy as np
import os, yaml

In [3]:
# RunRules for local computations
mpi = 4
omp = 2

rr = C.RunRules(omp_num_threads=omp,mpi=mpi)
code = C.QeCalculator(rr)
#code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


In [4]:
# RunRules for ismhpc
# The product of ntasks_per_node*cpus_per_task is equal to 32. 
# Many mpi are needed for better performances
nodes = 1
ntasks_per_node = 16
cpus_per_task=2
omp_num_threads=2

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='all12h',
                memory='125000',time='11:59:00',
                nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads)
code = C.QeCalculator(rr,activate_BeeOND=True) #,skip=False,clean_restart=False
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 16,
 'cpus_per_task': 2,
 'omp_num_threads': 2,
 'gpus_per_node': None,
 'memory': '125000',
 'time': '11:59:00',
 'partition': 'all12h',
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'executable': 'pw.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': True,
 'verbose': True}

# PHONON analysis of the bulk WSe$_2$

We compute the electron-phonon matrix elements using the QuantumESPRESSO package.

In [5]:
run_dir = 'PHONON'

## SCF and NSCF calculations

In [7]:
inp = I.PwInput(file='Pw_input_Marini/scf.in')
inp.set_pseudo_dir('../pseudos')
print(inp.convert_string())

&control
         calculation = 'scf'
              outdir = './'
              prefix = 'WSe2'
          pseudo_dir = '../pseudos'
           verbosity = 'high'
/&end
&system
           celldm(1) = 6.1983017192617
           celldm(3) = 3.95
             ecutwfc = 50
    force_symmorphic = .TRUE.
               ibrav = 4
            lspinorb = .true.
                 nat = 6
            noncolin = .true.
                ntyp = 2
/&end
&electrons
            conv_thr = 1.0d-6
      diago_full_acc = .false.
/&end
ATOMIC_SPECIES
  Se   32.065            Se_FR.upf
   W   183.84             W_FR.upf
ATOMIC_POSITIONS { crystal }
  W   0.3333330000   0.6666670000   0.2500000000
  W   0.6666670000   0.3333330000   0.7500000000
 Se   0.3333330000   0.6666670000   0.6211484930
 Se   0.6666670000   0.3333330000   0.1211484930
 Se   0.3333330000   0.6666670000   0.8788515070
 Se   0.6666670000   0.3333330000   0.3788515070
K_POINTS { automatic }
  9  9  1  1  1  0
